## Question 3

In [9]:
import tweepy
from tweepy import OAuthHandler

consumer_key = 'aZgjP0NZeM4iiBaQXJcyOKLJB'
consumer_secret = 'VRLyop5JLP8Kh91CMnE0rpOKUhKEtrrVTCOozHVOd8QEFx69XX'
access_token = '326266960-9RZItG8Q4FUER0BOX4eO8mrvsvrrDWzaWiJqoens'
access_secret = 'v6UHn9puTUv5OgcDwGqf3UxfCZv6XSJZ1gB8UkXOFrnQo'

auth = OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)

api = tweepy.API(auth)

In [19]:
import json

def process_or_store(tweet):
    print(json.dumps(tweet))

In [21]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    #print(status._json['user'])
    #print('###')
    process_or_store(status._json) 

{"created_at": "Sat Jan 20 16:06:12 +0000 2018", "id": 954746891954216966, "id_str": "954746891954216966", "text": "The greatest gift you can give someone is your time, your attention, your love, your concern", "truncated": false, "entities": {"hashtags": [], "symbols": [], "user_mentions": [], "urls": []}, "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 121598328, "id_str": "121598328", "name": "#-FOAMPASTEKiiD.!", "screen_name": "foampastekiid", "location": "C A L I F O R N I A", "description": "Do Numbers & Remain Humble \ud83d\ude4f\ud83c\udffd", "url": null, "entities": {"description": {"urls": []}}, "protected": false, "followers_count": 592866, "friends_count": 361797, "listed_count": 100, "created_at": "Wed Mar 10 00:48:16 +0000 2010", "favourites_count": 2101, "utc_off

In [58]:
from tweepy import Stream
from tweepy.streaming import StreamListener

JSON_FILE_NAME='usa_tweets.json'
MAX_NUMBER_OF_TWEETS = 1500
tweets_gathered = 0

class MyListener(StreamListener):
    
    def on_data(self, data):
        global tweets_gathered
        global json_name
        if MAX_NUMBER_OF_TWEETS > tweets_gathered:
            tweets_gathered += 1
            try:
                with open(JSON_FILE_NAME, 'a') as f:
                    f.write(data)
                    return True
            except BaseException as e:
                print("Error on_data: %s" % str(e))
            return True
        else:
            return False
 
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
# those coordinates are boxed that represents most of america - the country that we've found to have the most tweetings. 
twitter_stream.filter(locations=[-124.89,32.7,-93.07,48.99,
                                 -110.64,31.98,-84.43,47.01,
                                -99.04,26.29,-72.83,41.58,
                                -79.0,32.1,-70.6,43.69,
                                -75.3,41.06,-66.91,45.06])

Error on_data: 


KeyboardInterrupt: 

In [56]:
import json

with open(JSON_FILE_NAME, 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print

{
    "created_at": "Sat Jan 20 18:07:10 +0000 2018",
    "id": 954777334074273792,
    "id_str": "954777334074273792",
    "text": "why am i always hungry",
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 1644664255,
        "id_str": "1644664255",
        "name": "Taylinn",
        "screen_name": "PetersTia",
        "location": "Dodge City, KS",
        "url": null,
        "description": "@levistein6 \ud83d\udc95 #CamaroGang",
        "translator_type": "none",
        "protected": false,
        "verified": false,
        "followers_count": 671,
        "friends_count": 407,
        "listed_count": 5,
        "favourites_count": 18209,
        "statuses_count": 4331,
        "created_at": "Sun Aug

In [57]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'the',u'\u2019',u'\u2026','The',u'de',u'\xe9']

import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s.encode('ascii','ignore'))
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens